<a href="https://colab.research.google.com/github/mudogruer/LLMs/blob/main/LLama-7b-Zero-Shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U accelerate bitsandbytes bert-score evaluate

In [2]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quanty_type = "nf4",
    bnb_4bit_use_double_quanty = True,
)


In [3]:
# from huggingface_hub import login
# login()

In [4]:
model_name = "microsoft/phi-2"

from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code = True,

)

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype = torch.bfloat16,
    device_map = "auto"

)


2024-02-20 14:58:51.479782: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-20 14:58:51.479904: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-20 14:58:51.773805: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
prompt = "Instruct:"+"What is the heaviest element?"+"\nOutput:"

sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 100,
    temperature = 0.2,
    top_k = 2,
    top_p = 1.0,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [8]:
print(sequence[0]['generated_text'])

Instruct:What is the heaviest element?
Output: The heaviest element is oganesson.



In [9]:
sequence

[{'generated_text': 'Instruct:What is the heaviest element?\nOutput: The heaviest element is oganesson.\n'}]

In [10]:
prompt = "Instruct:"+"What is the gravity on mars?"+"\nOutput:"

sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 100,
    temperature = 0.2,
    top_k = 2,
    top_p = 1.0,
)
print(sequence[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Instruct:What is the gravity on mars?
Output: What is the gravity on Mars?



In [11]:
def Sequence(promt):
    sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 30,
    temperature = 0.1,
    top_k = 2,
    top_p = 1.0,
    pad_token_id=tokenizer.eos_token_id
    )
    return sequence

In [12]:
prompt = "Instruct:"+"Who invented mirror telescope?"+"\nOutput:"

print(Sequence(prompt)[0]['generated_text'])

Instruct:Who invented mirror telescope?
Output: The mirror telescope was invented by Galileo Galilei.



In [13]:
from datasets import load_dataset

dataset = load_dataset("sciq")

Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset sciq downloaded and prepared to /root/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 11679
    })
    validation: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
})

In [15]:
dataset_validation = load_dataset('sciq',split='validation[:20%]') #first %20 of validation data

In [16]:
dataset_validation

Dataset({
    features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
    num_rows: 200
})

In [17]:
answers = []
for question in dataset_validation["question"]:
    prompt = "Instruct:"+question+"\nOutput:"
    output_sequences = Sequence(prompt)[0]['generated_text']
    question_and_answer = output_sequences.split("\nOutput:")
    answer = question_and_answer[1]
    answers.append(answer)

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [18]:
from bert_score import score

# Assuming `answers` is your list of generated answers and `actual_answers` is the list of reference answers
# Both should be lists of strings

# Calculate BertScore
P, R, F1 = score(answers, dataset_validation["correct_answer"], lang='en', verbose=True)

# `P`, `R`, and `F1` are returned as tensors, so you might want to convert them to Python lists or take their mean
bertscore_precision = P.mean().item()
bertscore_recall = R.mean().item()
bertscore_f1 = F1.mean().item()

print(f"Precision: {bertscore_precision:.3f}")
print(f"Recall: {bertscore_recall:.3f}")
print(f"F1: {bertscore_f1:.3f}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 1.45 seconds, 137.98 sentences/sec
Precision: 0.828
Recall: 0.842
F1: 0.834


In [19]:
import pandas as pd
df = pd.DataFrame({"generated_answers":answers,"correct_answers":dataset_validation["correct_answer"]})

In [20]:
df.head(10)

,generated_answers,correct_answers
0,Charles Darwin\n,darwin
1,Each specific polypeptide has a unique linear...,amino
2,A frameshift mutation is a deletion or inserti...,nucleotides
3,Wetland.\n,wetland
4,The sun.\n,the sun
5,"Arteries, veins, and capillaries are examples...",blood vessels
6,The two general categories of biochemical rea...,catabolic and anabolic
7,The clay fractions of soils derived from gran...,volcanic ash
8,The brain.\n,brain
9,"The term ""rope"" can refer to a rope in a part...",lariat


In [21]:
from evaluate import load
import numpy as np
bertscore = load("bertscore")
predictions = answers
references = dataset_validation["correct_answer"]
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
print("precision: ",round(np.mean(list(results["precision"])),5))
print("recall: ",round(np.mean(list(results["recall"])),5))
print("f1: ",round(np.mean(list(results["f1"])),5))

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

precision:  0.70622
recall:  0.79844
f1:  0.74801
